<h1> Plots Flux Data


---

<h3> Jack Colwell and Sujan Pal
    
    Data from: loggers located at ATMOS

<h2> Imports

---

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from datetime import timedelta, date
import os
import glob
import requests
import wget

<h2>Storing the Downloaded Data Correctly

---

In [2]:
#This accesses the folder path that contains the downloaded data
folder_path = r'C:\Users\colwe\OneDrive\Desktop\Argonne\Fluxes'

#This list contains the months that the data is drawn from, in this case they are the same name as the folders in the directory specified above
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

folders = []

#Essentially, this loop collects the matching folders for each month and stores them in the folders list
for month in months:
    folders.append(sorted(glob.glob(folder_path + months +'/*')))
    
#Stores all the folders into one list
all_folders = [j for i in folders for j in i]

TypeError: can only concatenate str (not "list") to str

In [3]:
#Read all the flux data from the .csv files generated by smartflux
fluxdata = []
for folder in all_folders:
    try: 
        file = glob.glob(folder+'/output/eddypro_exp_full_output*')
        df = pd.read_csv(file[0],header=None, skiprows = 3)
        fluxdata.append(df)
        del df,file
    except:
        print("skip") # some folders have .tmp files and not csv, ignore those

NameError: name 'all_folders' is not defined

In [5]:
# add the column names to the data frame from any random single file
all_data = pd.concat(fluxdata, axis=0)
folder = all_folders[100] # 100 is random here
onefile = glob.glob(folder+'/output/eddypro_exp_full_output*')
line = pd.read_csv(onefile[0],header=1)
all_data.columns = line.columns

ValueError: No objects to concatenate

In [ ]:
# keeping only the data needed to plot fluxes
data = all_data[["date","time","ET","LE","H","co2_flux","h2o_flux","ch4_flux","air_temperature","air_pressure","specific_humiity\",\"RH\",\"VPD\",\"Tdew\",\"wind_speed\",\"wind_dir\",\"bowen_ratio\"]]"

In [ ]:
# Calculate hourly and daily values from half-hourly
data['Datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'])
data = data.set_index('Datetime')
data = data.drop(['date','time'], axis=1)
data_hourly = data.resample('H').mean()
data_daily = data.resample('D').mean()

In [ ]:
# Clean up data for calculations and plotting
# set unrealistic data to NaN
data_hourly.LE[data_hourly.LE<=-200] = np.NaN
data_hourly.LE[data_hourly.LE>= 600] = np.NaN
data_hourly.co2_flux[data_hourly.co2_flux<=-100] = np.NaN
data_hourly.co2_flux[data_hourly.co2_flux>= 100] = np.NaN
#data_hourly.ch4_flux[data_hourly.ch4_flux<=-100] = np.NaN
data_hourly.ch4_flux[data_hourly.ch4_flux>= 0.2] = np.NaN

In [ ]:
fig,ax = plt.subplots(figsize=(15,5))
ax.plot(data_hourly.index,data_hourly['LE'], label = 'LE')
ax.set_xlabel('Dates', fontsize =14)

ax.set_ylabel('Flux (W/m2)', color='black', fontsize =14)
ax.set_title('Latent heat flux at ATMOS', fontsize =16)
ax.legend(loc = 'upper left', frameon=False, fontsize=14)
plt.savefig('LHflux_ATMOS.png', dpi=300)

In [ ]:
fig,ax = plt.subplots(figsize=(15,5))
ax.plot(data_hourly.index,data_hourly['co2_flux'], label = 'CO2')
ax.set_xlabel('Dates', fontsize =14)
ax.set_ylabel('Flux (m-mol/m2/s)', color='black', fontsize =14)
ax.set_title('CO2 flux at ATMOS', fontsize =16)
ax.legend(loc = 'upper left', frameon=False, fontsize=14)
plt.savefig('CO2flux_ATMOS.png', dpi=300)

In [ ]:
fig,ax = plt.subplots(figsize=(15,5))
ax.plot(data_hourly.index,data_hourly['air_temperature']-273.15, label = 'Temp')
ax.set_xlabel('Dates', fontsize =14)
ax.set_ylabel('Deg C', color='black', fontsize =14)
ax.set_title('Air temperature at ATMOS', fontsize =16)
ax.legend(loc = 'upper left', frameon=False, fontsize=14)
plt.savefig('Temp_ATMOS.png', dpi=300)

In [ ]:
fig,ax = plt.subplots(figsize=(15,5))
ax.plot(data_hourly.index,data_hourly['ch4_flux'], label = 'CH4')
ax.set_xlabel('Dates', fontsize =14)
ax.set_ylabel('Flux (m-mol/m2/s)', color='black', fontsize =14)
ax.set_title('CH4 Flux at ATMOS', fontsize =16)
ax.legend(loc = 'upper left', frameon=False, fontsize=14)
plt.savefig('CH4flux_ATMOS.png', dpi=300)